Import the preprepared dataset, indexed by system_metadata_id, and renaming columns to replace the '.' with _ to avoid later errors.

In [ ]:
import requests
from urllib import parse
from pprint import pprint
import pandas as pd
import config
import numpy as np
import os, sys

if not os.path.exists('data/cleaned_opendoar_data.csv'):
    print('data/cleaned_opendoar_data.csv doesnt exist, please run international_opendoar.py to create it.')
    sys.exit()

input_data = 'data/cleaned_opendoar_data.csv'
df_all_data = pd.read_csv(input_data, header=0)
df_all_data.columns=df_all_data.columns.str.replace('.','_')
df_all_data.set_index('system_metadata_id', inplace=True)

display(df_all_data)

 

### Types of RIS

In [ ]:
''' Table of count of RIS types'''
vals = df_all_data.groupby("repository_metadata_software_name").size().sort_values(ascending=False)
display(vals)


&NewLine;  

This contains a lot of 'other' values, but `repository_metadata_software_name` can be used to replace the 'other' value for something more specific.

In [ ]:
display(df_all_data["repository_metadata_software_name"])

df_all_data.loc[df_all_data.repository_metadata_software_name == 'other', 'repository_metadata_software_name'] = df_all_data.repository_metadata_software_name_other


In [ ]:
display(df_all_data["repository_metadata_software_name"])


In [ ]:
display(df_all_data)

We can now redo the table with better detail.

In [ ]:
''' Table of count of RIS types'''
vals = df_all_data.groupby("repository_metadata_software_name").size().sort_values(ascending=False)
display(vals)

and dump to latex if needed.

In [ ]:
vals.to_latex()

In [ ]:
df_all_data.to_csv('./all_data.csv')